In [2]:
import sys
sys.path.append('../..')
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from config import ATLAS_CONNECTION_STRING
from pymongo import MongoClient
import pandas as pd

In [3]:
client = MongoClient(ATLAS_CONNECTION_STRING)
osu_db = client['osu_random_db']

In [4]:
#list of sample users' ids
cursor = osu_db["week_9_sample_users"].find({},{"users":1})
l = []
for el in cursor:
    l.append(el)
Ids = list(map(lambda x: x["users"], l))[0]
# print((Ids))



In [5]:
# work on this
collection = osu_db["osu_scores_high"]
df1=pd.DataFrame(list(collection.find(
        {'enabled_mods':0,"user_id" : {"$in" : Ids}},{'maxcombo':True,'count50':True,'count100':True,'count300':True,
    'countmiss':True,'countgeki':True,'countkatu':True,'beatmap_id':True}
    )
))
df1.head()

,_id,beatmap_id,maxcombo,count50,count100,count300,countmiss,countgeki,countkatu
0,13837747,110,140,7,41,216,3,13,17
1,13837087,194,133,0,5,87,0,22,4
2,28474531,207,103,2,26,180,11,97,19
3,7114604,279,71,0,5,40,0,20,5
4,7773047,280,77,3,19,75,2,17,9


In [6]:
# df1.describe()
list1=list(df1['beatmap_id'])
# print(list1)

In [11]:
collection2 = osu_db["osu_beatmaps"]
df2=pd.DataFrame(list(collection2.find(
        {"_id" : {"$in" : list1}}
    )
))
df2.head()

,_id,approved,beatmapset_id,bpm,checksum,countNormal,countSlider,countSpinner,countTotal,deleted_at,...,orphaned,passcount,playcount,playmode,score_version,total_length,user_id,version,youtube_preview,mlpp
0,53,1,3,172.000,1d23c37a2fda439be752ae2bca06c0cd,67,15,1,100,None,...,0,47715,100953,0,1,83,141,-Crusin-,None,{'order': 0}
1,54,1,3,172.000,e6f48da7f132c909accc677b77f231b8,102,23,1,151,None,...,0,34918,156874,0,1,83,141,-Hardrock-,None,{'order': 1}
2,55,1,3,172.000,a5372216d0902bacc7eb081e15e36bb9,95,23,1,144,None,...,0,35348,109145,0,1,83,141,-Sweatin-,None,{'order': 2}
3,80,1,18,136.016,ea43709e1e9056879e59860b286895be,270,0,0,341,None,...,0,292885,2009768,0,1,185,0,Insane,None,{'order': 4}
4,87,1,24,153.001,f8b821b546bec6f705276821dc99fb10,278,55,0,388,None,...,0,63578,260874,0,1,227,2,Easy,None,{'order': 6}
